# Predictions & Forecasting - AI/ML Market Analysis

This notebook generates comprehensive predictions and forecasts for AI market trends using trained models.

## Objectives:
- Generate revenue forecasts with confidence intervals
- Predict AI adoption rates and market penetration
- Project job market impacts and transformations
- Create scenario analysis and what-if simulations
- Visualize predictions vs actual data
- Provide actionable forecast insights

## 1. Import Libraries and Load Models

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Model loading and utilities
import joblib
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Statistical analysis
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load data and check for saved models
processed_dir = Path('../data/processed')
models_dir = Path('../models')
results_dir = Path('../results')

market_df = pd.read_csv(processed_dir / 'ai_market_engineered.csv')

print("✅ Libraries imported and data loaded!")
print(f"Market data shape: {market_df.shape}")
print(f"Models directory exists: {models_dir.exists()}")
print(f"Results directory exists: {results_dir.exists()}")

## 2. Load Trained Models and Results

In [ ]:
# Load model comparison results
def load_best_models():
    """
    Load the best performing models for each target
    """
    best_models = {}
    model_performance = {}
    
    # Check for model comparison files
    comparison_files = list(results_dir.glob('model_comparison_*.csv'))
    
    if comparison_files:
        print("📊 Loading model performance data...")
        
        for file in comparison_files:
            # Extract target name from filename
            target_col = file.stem.replace('model_comparison_', '')
            
            try:
                comparison_df = pd.read_csv(file)
                
                if 'Train_R2' in comparison_df.columns:
                    # Find best model
                    best_idx = comparison_df['Train_R2'].idxmax()
                    best_model_name = comparison_df.loc[best_idx, 'Model'].lower().replace(' ', '_')
                    best_r2 = comparison_df.loc[best_idx, 'Train_R2']
                    
                    model_performance[target_col] = {
                        'best_model': best_model_name,
                        'best_r2': best_r2,
                        'comparison_df': comparison_df
                    }
                    
                    print(f"   ✅ {target_col}: Best model = {best_model_name} (R² = {best_r2:.3f})")
            
            except Exception as e:
                print(f"   ⚠️ Could not load {file}: {e}")
    
    else:
        print("⚠️ No model comparison files found. Will create basic predictions.")
    
    return model_performance

# Load model performance data
model_performance = load_best_models()

## 3. Generate Revenue Forecasts

In [ ]:
# Generate revenue forecasts with confidence intervals
def generate_revenue_forecasts(df, forecast_years=5):
    """
    Generate comprehensive revenue forecasts
    """
    revenue_col = 'ai_software_revenue_in_billions'
    year_col = 'year'
    
    if revenue_col not in df.columns or year_col not in df.columns:
        print(f"❌ Required columns not found: {revenue_col}, {year_col}")
        return None
    
    # Prepare historical data
    historical_data = df[[year_col, revenue_col]].dropna()
    historical_data = historical_data.sort_values(year_col)
    
    print(f"💰 REVENUE FORECASTING ANALYSIS")
    print("=" * 50)
    print(f"Historical data points: {len(historical_data)}")
    print(f"Year range: {historical_data[year_col].min():.0f} - {historical_data[year_col].max():.0f}")
    print(f"Revenue range: ${historical_data[revenue_col].min():.1f}B - ${historical_data[revenue_col].max():.1f}B")
    
    # Method 1: Linear trend extrapolation
    X = historical_data[year_col].values.reshape(-1, 1)
    y = historical_data[revenue_col].values
    
    from sklearn.linear_model import LinearRegression
    linear_model = LinearRegression()
    linear_model.fit(X, y)
    
    # Method 2: Exponential growth model
    # Fit: y = a * exp(b * x)
    log_y = np.log(y)
    exp_model = LinearRegression()
    exp_model.fit(X, log_y)
    
    # Method 3: Polynomial trend (degree 2)
    from sklearn.preprocessing import PolynomialFeatures
    poly_features = PolynomialFeatures(degree=2)
    X_poly = poly_features.fit_transform(X)
    poly_model = LinearRegression()
    poly_model.fit(X_poly, y)
    
    # Generate forecasts
    future_years = np.arange(
        historical_data[year_col].max() + 1,
        historical_data[year_col].max() + forecast_years + 1
    ).reshape(-1, 1)
    
    # Linear forecast
    linear_forecast = linear_model.predict(future_years)
    
    # Exponential forecast
    exp_forecast = np.exp(exp_model.predict(future_years))
    
    # Polynomial forecast
    future_poly = poly_features.transform(future_years)
    poly_forecast = poly_model.predict(future_poly)
    
    # Calculate confidence intervals (using historical residuals)
    linear_residuals = y - linear_model.predict(X)
    residual_std = np.std(linear_residuals)
    
    # 95% confidence interval
    confidence_interval = 1.96 * residual_std
    
    # Create forecast summary
    forecast_df = pd.DataFrame({
        'year': future_years.flatten(),
        'linear_forecast': linear_forecast,
        'exponential_forecast': exp_forecast,
        'polynomial_forecast': poly_forecast,
        'ensemble_forecast': (linear_forecast + exp_forecast + poly_forecast) / 3,
        'ci_lower': linear_forecast - confidence_interval,
        'ci_upper': linear_forecast + confidence_interval
    })
    
    print(f"\n🔮 REVENUE FORECASTS ({forecast_years} years):")
    for _, row in forecast_df.iterrows():
        year = int(row['year'])
        ensemble = row['ensemble_forecast']
        ci_lower = row['ci_lower']
        ci_upper = row['ci_upper']
        print(f"   {year}: ${ensemble:.1f}B (95% CI: ${ci_lower:.1f}B - ${ci_upper:.1f}B)")
    
    return forecast_df, historical_data, {
        'linear_model': linear_model,
        'exp_model': exp_model,
        'poly_model': poly_model,
        'poly_features': poly_features
    }

# Generate revenue forecasts
revenue_forecast, revenue_historical, revenue_models = generate_revenue_forecasts(market_df)

## 4. AI Adoption Rate Predictions

In [ ]:
# Predict AI adoption rates
def predict_adoption_rates(df, forecast_years=5):
    """
    Predict AI adoption rates using logistic growth model
    """
    adoption_col = 'ai_adoption'
    year_col = 'year'
    
    if adoption_col not in df.columns:
        print(f"❌ Adoption column not found: {adoption_col}")
        return None
    
    # Prepare data
    adoption_data = df[[year_col, adoption_col]].dropna()
    adoption_data = adoption_data.sort_values(year_col)
    
    print(f"📈 AI ADOPTION RATE PREDICTIONS")
    print("=" * 50)
    
    # Method 1: Logistic growth model
    # Assume market saturation at 95%
    max_adoption = 95
    
    # Fit logistic curve: y = L / (1 + exp(-k(x-x0)))
    years = adoption_data[year_col].values
    adoption_rates = adoption_data[adoption_col].values
    
    # Simple logistic approximation
    from scipy.optimize import curve_fit
    
    def logistic_func(x, L, k, x0):
        return L / (1 + np.exp(-k * (x - x0)))
    
    try:
        # Initial guess
        p0 = [max_adoption, 0.3, years.mean()]
        popt, pcov = curve_fit(logistic_func, years, adoption_rates, p0=p0, maxfev=1000)
        
        L_opt, k_opt, x0_opt = popt
        
        # Generate forecasts
        future_years = np.arange(
            years.max() + 1,
            years.max() + forecast_years + 1
        )
        
        logistic_forecast = logistic_func(future_years, L_opt, k_opt, x0_opt)
        
        print(f"📊 Logistic Growth Parameters:")
        print(f"   Maximum adoption (L): {L_opt:.1f}%")
        print(f"   Growth rate (k): {k_opt:.3f}")
        print(f"   Inflection point (x0): {x0_opt:.1f}")
        
    except Exception as e:
        print(f"⚠️ Logistic curve fitting failed: {e}")
        # Fallback to linear trend
        from sklearn.linear_model import LinearRegression
        linear_model = LinearRegression()
        linear_model.fit(years.reshape(-1, 1), adoption_rates)
        
        future_years = np.arange(
            years.max() + 1,
            years.max() + forecast_years + 1
        )
        
        logistic_forecast = linear_model.predict(future_years.reshape(-1, 1))
        # Cap at reasonable maximum
        logistic_forecast = np.clip(logistic_forecast, 0, 95)
    
    # Method 2: Linear extrapolation with constraints
    linear_model = LinearRegression()
    linear_model.fit(years.reshape(-1, 1), adoption_rates)
    linear_forecast = linear_model.predict(future_years.reshape(-1, 1))
    linear_forecast = np.clip(linear_forecast, 0, 95)  # Realistic bounds
    
    # Create adoption forecast DataFrame
    adoption_forecast_df = pd.DataFrame({
        'year': future_years,
        'logistic_forecast': logistic_forecast,
        'linear_forecast': linear_forecast,
        'ensemble_forecast': (logistic_forecast + linear_forecast) / 2
    })
    
    print(f"\n🎯 ADOPTION RATE FORECASTS:")
    for _, row in adoption_forecast_df.iterrows():
        year = int(row['year'])
        ensemble = row['ensemble_forecast']
        print(f"   {year}: {ensemble:.1f}% adoption rate")
    
    return adoption_forecast_df, adoption_data

# Generate adoption predictions
adoption_forecast, adoption_historical = predict_adoption_rates(market_df)

## 5. Job Market Impact Projections

In [ ]:
# Project job market impacts
def project_job_market_impact(df, forecast_years=5):
    """
    Project job market transformations
    """
    jobs_eliminated_col = 'estimated_jobs_eliminated_by_ai_millions'
    jobs_created_col = 'estimated_new_jobs_created_by_ai_millions'
    year_col = 'year'
    
    print(f"👥 JOB MARKET IMPACT PROJECTIONS")
    print("=" * 50)
    
    if all(col in df.columns for col in [jobs_eliminated_col, jobs_created_col, year_col]):
        # Prepare job data
        job_data = df[[year_col, jobs_eliminated_col, jobs_created_col]].dropna()
        job_data = job_data.sort_values(year_col)
        
        years = job_data[year_col].values
        eliminated = job_data[jobs_eliminated_col].values
        created = job_data[jobs_created_col].values
        net_impact = created - eliminated
        
        print(f"Historical job data points: {len(job_data)}")
        print(f"Current net job impact (2025): {net_impact[-1]:.1f}M jobs")
        
        # Fit trends for job elimination and creation
        from sklearn.linear_model import LinearRegression
        
        # Jobs eliminated trend
        elim_model = LinearRegression()
        elim_model.fit(years.reshape(-1, 1), eliminated)
        
        # Jobs created trend
        created_model = LinearRegression()
        created_model.fit(years.reshape(-1, 1), created)
        
        # Generate forecasts
        future_years = np.arange(
            years.max() + 1,
            years.max() + forecast_years + 1
        )
        
        future_eliminated = elim_model.predict(future_years.reshape(-1, 1))
        future_created = created_model.predict(future_years.reshape(-1, 1))
        future_net_impact = future_created - future_eliminated
        
        # Create job forecast DataFrame
        job_forecast_df = pd.DataFrame({
            'year': future_years,
            'jobs_eliminated_forecast': future_eliminated,
            'jobs_created_forecast': future_created,
            'net_job_impact_forecast': future_net_impact
        })
        
        print(f"\n🔮 JOB MARKET FORECASTS:")
        for _, row in job_forecast_df.iterrows():
            year = int(row['year'])
            eliminated = row['jobs_eliminated_forecast']
            created = row['jobs_created_forecast']
            net = row['net_job_impact_forecast']
            print(f"   {year}: {eliminated:.1f}M eliminated, {created:.1f}M created, Net: {net:+.1f}M")
        
        # Calculate key insights
        total_net_impact = future_net_impact.sum()
        avg_annual_creation = future_created.mean()
        avg_annual_elimination = future_eliminated.mean()
        
        print(f"\n📊 KEY JOB MARKET INSIGHTS:")
        print(f"   📈 Average annual job creation: {avg_annual_creation:.1f}M")
        print(f"   📉 Average annual job elimination: {avg_annual_elimination:.1f}M")
        print(f"   ⚖️ Total net impact ({forecast_years} years): {total_net_impact:+.1f}M jobs")
        
        return job_forecast_df, job_data
    
    else:
        print("❌ Job market columns not found")
        return None, None

# Generate job market projections
job_forecast, job_historical = project_job_market_impact(market_df)

## 6. Comprehensive Forecast Visualization

In [ ]:
# Create comprehensive forecast visualization
def create_forecast_dashboard(revenue_hist, revenue_forecast, adoption_hist, adoption_forecast, job_hist, job_forecast):
    """
    Create interactive forecast dashboard
    """
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'AI Revenue Forecast with Confidence Intervals',
            'AI Adoption Rate Predictions',
            'Job Market Impact Projections',
            'Market Growth Scenarios'
        ),
        vertical_spacing=0.1
    )
    
    # 1. Revenue forecast
    if revenue_hist is not None and revenue_forecast is not None:
        # Historical data
        fig.add_trace(
            go.Scatter(
                x=revenue_hist['year'],
                y=revenue_hist['ai_software_revenue_in_billions'],
                mode='lines+markers',
                name='Historical Revenue',
                line=dict(color='blue', width=3)
            ),
            row=1, col=1
        )
        
        # Forecast
        fig.add_trace(
            go.Scatter(
                x=revenue_forecast['year'],
                y=revenue_forecast['ensemble_forecast'],
                mode='lines+markers',
                name='Revenue Forecast',
                line=dict(color='red', width=3, dash='dash')
            ),
            row=1, col=1
        )
        
        # Confidence interval
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([revenue_forecast['year'], revenue_forecast['year'][::-1]]),
                y=np.concatenate([revenue_forecast['ci_upper'], revenue_forecast['ci_lower'][::-1]]),
                fill='toself',
                fillcolor='rgba(255,0,0,0.2)',
                line=dict(color='rgba(255,255,255,0)'),
                name='95% Confidence Interval',
                showlegend=True
            ),
            row=1, col=1
        )
    
    # 2. Adoption forecast
    if adoption_hist is not None and adoption_forecast is not None:
        # Historical adoption
        fig.add_trace(
            go.Scatter(
                x=adoption_hist['year'],
                y=adoption_hist['ai_adoption'],
                mode='lines+markers',
                name='Historical Adoption',
                line=dict(color='green', width=3)
            ),
            row=1, col=2
        )
        
        # Adoption forecast
        fig.add_trace(
            go.Scatter(
                x=adoption_forecast['year'],
                y=adoption_forecast['ensemble_forecast'],
                mode='lines+markers',
                name='Adoption Forecast',
                line=dict(color='orange', width=3, dash='dash')
            ),
            row=1, col=2
        )
    
    # 3. Job impact
    if job_hist is not None and job_forecast is not None:
        # Historical job impact
        fig.add_trace(
            go.Scatter(
                x=job_hist['year'],
                y=job_hist['estimated_new_jobs_created_by_ai_millions'],
                mode='lines+markers',
                name='Jobs Created (Historical)',
                line=dict(color='green', width=2)
            ),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=job_hist['year'],
                y=job_hist['estimated_jobs_eliminated_by_ai_millions'],
                mode='lines+markers',
                name='Jobs Eliminated (Historical)',
                line=dict(color='red', width=2)
            ),
            row=2, col=1
        )
        
        # Job forecast
        fig.add_trace(
            go.Scatter(
                x=job_forecast['year'],
                y=job_forecast['jobs_created_forecast'],
                mode='lines+markers',
                name='Jobs Created (Forecast)',
                line=dict(color='green', width=2, dash='dash')
            ),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=job_forecast['year'],
                y=job_forecast['jobs_eliminated_forecast'],
                mode='lines+markers',
                name='Jobs Eliminated (Forecast)',
                line=dict(color='red', width=2, dash='dash')
            ),
            row=2, col=1
        )
    
    # 4. Scenario comparison
    if revenue_forecast is not None:
        fig.add_trace(
            go.Scatter(
                x=revenue_forecast['year'],
                y=revenue_forecast['linear_forecast'],
                mode='lines',
                name='Conservative Scenario',
                line=dict(color='blue', width=2)
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=revenue_forecast['year'],
                y=revenue_forecast['exponential_forecast'],
                mode='lines',
                name='Optimistic Scenario',
                line=dict(color='green', width=2)
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=revenue_forecast['year'],
                y=revenue_forecast['ensemble_forecast'],
                mode='lines',
                name='Realistic Scenario',
                line=dict(color='orange', width=3)
            ),
            row=2, col=2
        )
    
    fig.update_layout(
        height=800,
        title_text="AI Market Forecasting Dashboard",
        showlegend=True
    )
    
    fig.show()

# Create forecast dashboard
if all(x is not None for x in [revenue_historical, revenue_forecast, adoption_historical, adoption_forecast]):
    create_forecast_dashboard(
        revenue_historical, revenue_forecast,
        adoption_historical, adoption_forecast,
        job_historical if 'job_historical' in locals() else None,
        job_forecast
    )
else:
    print("⚠️ Some forecast data not available for dashboard")

## 7. Scenario Analysis and What-If Simulations

In [ ]:
# Scenario analysis
def perform_scenario_analysis(base_forecast, scenarios):
    """
    Perform what-if scenario analysis
    """
    print(f"🎭 SCENARIO ANALYSIS")
    print("=" * 50)
    
    scenario_results = {}
    
    if revenue_forecast is not None:
        base_revenue = revenue_forecast['ensemble_forecast'].values
        
        # Define scenarios
        scenarios = {
            'Conservative': 0.7,   # 30% slower growth
            'Realistic': 1.0,      # Base case
            'Optimistic': 1.3,     # 30% faster growth
            'AI Winter': 0.4,      # Significant slowdown
            'AI Boom': 1.8         # Exponential acceleration
        }
        
        print("💰 REVENUE SCENARIOS (2030 projection):")
        
        for scenario_name, multiplier in scenarios.items():
            scenario_revenue = base_revenue * multiplier
            final_value = scenario_revenue[-1]
            
            scenario_results[scenario_name] = {
                'multiplier': multiplier,
                'forecast': scenario_revenue,
                'final_value': final_value
            }
            
            print(f"   📊 {scenario_name}: ${final_value:.1f}B (×{multiplier})")
        
        # Visualize scenarios
        fig = go.Figure()
        
        colors = {
            'Conservative': 'blue',
            'Realistic': 'green',
            'Optimistic': 'orange',
            'AI Winter': 'red',
            'AI Boom': 'purple'
        }
        
        for scenario_name, data in scenario_results.items():
            fig.add_trace(
                go.Scatter(
                    x=revenue_forecast['year'],
                    y=data['forecast'],
                    mode='lines+markers',
                    name=f"{scenario_name} Scenario",
                    line=dict(color=colors[scenario_name], width=3)
                )
            )
        
        fig.update_layout(
            title='AI Revenue Scenario Analysis (2026-2030)',
            xaxis_title='Year',
            yaxis_title='Revenue (Billions USD)',
            height=600
        )
        
        fig.show()
        
        return scenario_results
    
    return None

# Perform scenario analysis
if revenue_forecast is not None:
    scenario_results = perform_scenario_analysis(revenue_forecast, {})
else:
    print("⚠️ Revenue forecast not available for scenario analysis")
    scenario_results = None

## 8. Prediction Accuracy and Model Diagnostics

In [ ]:
# Model diagnostics and accuracy assessment
def assess_prediction_accuracy(historical_data, models, target_col):
    """
    Assess prediction accuracy using historical data
    """
    print(f"🎯 PREDICTION ACCURACY ASSESSMENT")
    print("=" * 50)
    
    if len(historical_data) < 4:
        print("⚠️ Insufficient historical data for accuracy assessment")
        return None
    
    # Use last 30% of data for validation
    split_idx = int(len(historical_data) * 0.7)
    train_data = historical_data.iloc[:split_idx]
    test_data = historical_data.iloc[split_idx:]
    
    if len(test_data) == 0:
        print("⚠️ No test data available")
        return None
    
    # Retrain on training data and predict test period
    X_train = train_data['year'].values.reshape(-1, 1)
    y_train = train_data[target_col].values
    X_test = test_data['year'].values.reshape(-1, 1)
    y_test = test_data[target_col].values
    
    # Linear model for accuracy assessment
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    # Calculate accuracy metrics
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    print(f"📊 Validation Results:")
    print(f"   R² Score: {r2:.3f}")
    print(f"   RMSE: {rmse:.2f}")
    print(f"   MAE: {mae:.2f}")
    print(f"   MAPE: {mape:.1f}%")
    
    # Residual analysis
    residuals = y_test - y_pred
    
    print(f"\n🔍 Residual Analysis:")
    print(f"   Mean residual: {residuals.mean():.3f}")
    print(f"   Residual std: {residuals.std():.3f}")
    print(f"   Max absolute error: {np.abs(residuals).max():.2f}")
    
    return {
        'r2': r2,
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'residuals': residuals,
        'y_test': y_test,
        'y_pred': y_pred
    }

# Assess accuracy for revenue predictions
if revenue_historical is not None:
    revenue_accuracy = assess_prediction_accuracy(
        revenue_historical, 
        revenue_models, 
        'ai_software_revenue_in_billions'
    )
else:
    revenue_accuracy = None

## 9. Save Predictions and Forecasts

In [ ]:
# Save all predictions and forecasts
predictions_dir = Path('../results/predictions')
predictions_dir.mkdir(exist_ok=True)

print("💾 SAVING PREDICTIONS AND FORECASTS")
print("=" * 50)

# Save revenue forecasts
if revenue_forecast is not None:
    revenue_forecast.to_csv(predictions_dir / 'revenue_forecasts.csv', index=False)
    print("   ✅ Revenue forecasts saved")

# Save adoption forecasts
if adoption_forecast is not None:
    adoption_forecast.to_csv(predictions_dir / 'adoption_forecasts.csv', index=False)
    print("   ✅ Adoption forecasts saved")

# Save job market forecasts
if job_forecast is not None:
    job_forecast.to_csv(predictions_dir / 'job_market_forecasts.csv', index=False)
    print("   ✅ Job market forecasts saved")

# Save scenario analysis
if scenario_results is not None:
    scenario_df = pd.DataFrame({
        'scenario': list(scenario_results.keys()),
        'multiplier': [data['multiplier'] for data in scenario_results.values()],
        'final_value_2030': [data['final_value'] for data in scenario_results.values()]
    })
    scenario_df.to_csv(predictions_dir / 'scenario_analysis.csv', index=False)
    print("   ✅ Scenario analysis saved")

# Create prediction summary
prediction_summary = {
    'prediction_timestamp': pd.Timestamp.now().isoformat(),
    'forecast_horizon': '5 years (2026-2030)',
    'models_used': list(model_performance.keys()) if model_performance else ['linear_trend'],
    'accuracy_metrics': revenue_accuracy if revenue_accuracy else 'Not available'
}

# Save summary
summary_df = pd.DataFrame([prediction_summary])
summary_df.to_csv(predictions_dir / 'prediction_summary.csv', index=False)

print(f"\n📋 PREDICTION SUMMARY:")
print(f"   🎯 Forecasts generated for: Revenue, Adoption, Job Impact")
print(f"   📅 Forecast horizon: 2026-2030 (5 years)")
print(f"   🔮 Scenarios analyzed: 5 different market conditions")
print(f"   📊 Prediction accuracy: {revenue_accuracy['mape']:.1f}% MAPE" if revenue_accuracy else "   📊 Prediction accuracy: To be validated")

print("\n🚀 NEXT STEPS:")
print("   1. Move to 07_final_report.ipynb for comprehensive analysis")
print("   2. Combine all insights into executive summary")
print("   3. Create presentation-ready visualizations")

print("\n✅ Predictions & Forecasting Phase Complete!")